In [1]:
import sys
sys.path.append('../py')

%pylab inline
import os
import cPickle as pk
from pymongo import MongoClient
from itertools import groupby
from datetime import datetime, timedelta
from utils import read_tweets_as_time_series
import pandas

Populating the interactive namespace from numpy and matplotlib


In [2]:
count_file_name = '/Users/finkcr1/data/bbog-topic-analysis/monthly_user_tweet_counts.pickle'
if not os.path.exists(count_file_name):
    client = MongoClient('finkcr1-ls1')
    db = client.get_database('naija_tweets')
    pipeline = [
        {
            '$match':{
                'account_id':{
                    '$in':monitored_users},'time_stamp':{'$gte':datetime(2013,1,1)}, 
                           'time_stamp':{'$lt':datetime(2015,11,1)}
            }
        }, 
        {
            '$group':{
                '_id':{
                    'account_id':'$account_id','year':{
                        '$year':'$time_stamp'},'month':{'$month':'$time_stamp'}},'count':{'$sum':1}
            }
        }
    ]
    monthy_user_tweet_counts = [u for u in db.get_collection('tweet').aggregate(pipeline, allowDiskUse=True)]
    
    with open('/Users/finkcr1/data/bbog-topic-analysis/monthly_user_tweet_counts.pickle','wb') as f:
        pk.dump(monthy_user_tweet_counts, f)
else:
    with open('/Users/finkcr1/data/bbog-topic-analysis/monthly_user_tweet_counts.pickle','rb') as f:
        monthy_user_tweet_counts = pk.load(f)

In [38]:
bbog_tweets = read_tweets_as_time_series(
    '/Users/finkcr1/data/bbog-topic-analysis/chibok-girls-tags-20161213-results.txt')
bbog_users = {t[1] for t in bbog_tweets if t[0] < datetime(2014,5,15)}

In [37]:
# Read in VAW tweets
topical_tweets1 = read_tweets_as_time_series(
    '/Volumes/redd_programs/Minerva/bbog-topic-analysis/retrieval-results/vaw-20131001-20140415-bim-query-30-100-10000-results.txt')
topical_users1 = {t[1] for t in topical_tweets1}

print 'Set 1: We have %d VAW tweets from %d users from %s to %s' % (
    len(topical_tweets1), len(topical_users1), topical_tweets1[0][0].date(), topical_tweets1[-1][0].date())

# Read in VAW tweets
topical_tweets2 = read_tweets_as_time_series(
    '/Volumes/redd_programs/Minerva/bbog-topic-analysis/retrieval-results/vaw-20140515-20141101-bim-query-50-100-10000-results.txt')
topical_users2 = {t[1] for t in topical_tweets2}

print 'Set 2: We have %d VAW tweets from %d users from %s to %s' % (
    len(topical_tweets2), len(topical_users2), topical_tweets2[0][0].date(), topical_tweets2[-1][0].date())

topical_users2_only = topical_users2.difference(topical_users1)
topical_users1_only = topical_users1.difference(topical_users2)

print '%d topical users in set 2 that were not in set 1' % len(topical_users2_only)
print '%d topical users in set 1 that were not in set 2' % len(topical_users1_only)

Set 1: We have 23562 VAW tweets from 9320 users from 2013-10-01 to 2014-04-15
Set 2: We have 34809 VAW tweets from 12714 users from 2014-05-15 to 2014-11-01
10845 topical users in set 2 that were not in set 1
7451 topical users in set 1 that were not in set 2


In [39]:
active_users = set()

func = lambda x: x['_id']['account_id']

for k, v in groupby(sorted(monthy_user_tweet_counts, key=func), func):
    account_id = k
    monthly_counts = list(v)
    months = set(sorted([datetime(u['_id']['year'],u['_id']['month'], 1) for u in monthly_counts]))
    if (len([m for m in months if datetime(2013,10,1) <= m < datetime(2014,5,15)]) > 1 
    and len([m for m in months if datetime(2014,5,15) <= m < datetime(2014,10,1)]) > 1 
    and datetime(2014,1,1) in months):
        active_users.add(k)


print "We have %d active monitored users." % len(active_users)

We have 25067 active monitored users.


In [40]:
monitored_topical_users1 = active_users.intersection(topical_users1)
monitored_non_topical_users1 = active_users.difference(topical_users1)


print "Set 1: We have %d monitored topical users and %d non-topical users" % (len(monitored_topical_users1), 
                                                                      len(monitored_non_topical_users1))

monitored_topical_users1_only = topical_users1_only.intersection(active_users)

print "%d topical users active in set 1 only" % len(monitored_topical_users1_only)

Set 1: We have 2796 monitored topical users and 22271 non-topical users
1684 topical users active in set 1 only


In [41]:
monitored_topical_users2 = active_users.intersection(topical_users2)
monitored_non_topical_users2 = active_users.difference(topical_users2)

print "Set 2: We have %d monitored topical users and %d non-topical users" % (len(monitored_topical_users2), 
                                                                      len(monitored_non_topical_users2))

monitored_topical_users2_only = topical_users2_only.intersection(active_users)

print "%d topical users active in set 2 only" % len(monitored_topical_users2_only)

Set 2: We have 3958 monitored topical users and 21109 non-topical users
2846 topical users active in set 2 only


In [42]:
monitored_bbog_users = bbog_users.intersection(active_users)
monitored_non_bbog_users = active_users.difference(bbog_users)

In [43]:
# partition users into sets
not_vaw1 = monitored_non_topical_users1
vaw1 = monitored_topical_users1
not_bbog = monitored_non_bbog_users
bbog = monitored_bbog_users
not_vaw2 = monitored_non_topical_users2
vaw2 = monitored_topical_users2
vaw2_and_not_vaw1 = vaw2.difference(vaw1)

not_vaw1_and_not_bbog = not_vaw1.intersection(not_bbog)
not_vaw1_and_bbog = not_vaw1.intersection(bbog)
vaw1_and_not_bbog = vaw1.intersection(not_bbog)
vaw1_and_bbog = vaw1.intersection(bbog)

N = float(len(active_users))

In [44]:
def get_conditional_probability(set1, set2, N):
    return "%4.2f (%d)" % ((len(set1.intersection(set2))/N)/(len(set2)/N), len(set1.intersection(set2)))

In [45]:
table1 = pandas.DataFrame(index=['BBOG'], columns=['Not VAW1','VAW1'])
table1.loc['BBOG']['Not VAW1'] = get_conditional_probability(bbog, not_vaw1, N)
table1.loc['BBOG']['VAW1'] = get_conditional_probability(bbog, vaw1, N)
table1

,Not VAW1,VAW1
BBOG,0.55 (12153),0.80 (2235)


In [46]:
table2 = pandas.DataFrame(index=['VAW2'],
                          columns=['Not VAW1', 'VAW1','Not BBOG','BBOG', 'Not VAW1 and Not BBOG', 'Not VAW1 and BBOG', 'VAW1 and Not BBOG',
                                 'VAW1 and BBOG'])

table2.loc['VAW2']['Not VAW1'] = get_conditional_probability(vaw2, not_vaw1, N)
table2.loc['VAW2']['VAW1'] = get_conditional_probability(vaw2, vaw1, N)
table2.loc['VAW2']['Not BBOG'] = get_conditional_probability(vaw2, not_bbog, N)
table2.loc['VAW2']['BBOG'] = get_conditional_probability(vaw2, bbog, N)
table2.loc['VAW2']['Not VAW1 and Not BBOG'] = get_conditional_probability(vaw2, not_vaw1_and_not_bbog, N)
table2.loc['VAW2']['Not VAW1 and BBOG'] = get_conditional_probability(vaw2, not_vaw1_and_bbog, N)
table2.loc['VAW2']['VAW1 and Not BBOG'] = get_conditional_probability(vaw2, vaw1_and_not_bbog, N)
table2.loc['VAW2']['VAW1 and BBOG'] = get_conditional_probability(vaw2, vaw1_and_bbog, N)

table2

,Not VAW1,VAW1,Not BBOG,BBOG,Not VAW1 and Not BBOG,Not VAW1 and BBOG,VAW1 and Not BBOG,VAW1 and BBOG
VAW2,0.13 (2846),0.40 (1112),0.10 (1015),0.20 (2943),0.09 (896),0.16 (1950),0.21 (119),0.44 (993)
